# Сигма-алгебра

Сигма-алгебра (σ-алгебра) - множество событий F, которое соблюдает следующие условия:
\
 1. Ω ∈ F
 2. Если A ∈ F, то ¬A ∈ F
 3. Если A₁, A₂, ... ∈ F, то A₁ U A₂ U ... ∈ F (отличается от алгебры событий тем, количество событий может быть счетным, а не конечным)

Наименьшая сигма-алгебра

Полная система событий
